<a href="https://colab.research.google.com/github/steffenmodest/Python_Examples/blob/main/M_Net_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import matplotlib.pyplot as plt

rs = 23  # fester Random Seed

Load Data

In [34]:
from tensorflow.keras.datasets import fashion_mnist, mnist

# Fashion MNIST Daten laden, wir wollen nur die Trainingsdaten und verwerfen die Testdaten
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


Preprocess data

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
X = X_train.reshape(len(X_train), -1).astype('float64')

# Dimensionen um den Mittelpunkt zentrieren
preproccessing = StandardScaler()
X = preproccessing.fit_transform(X)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_train.shape, np.mean(X_train), np.std(X_train)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

Originaldaten:
Shape: (60000, 28, 28), Mean: 33.318421, STD: 78.567490
Vorbereitete Daten:
Shape: (60000, 784), Mean: -0.000000, STD: 0.956316


Create Hidden Activations

In [36]:
H = np.random.rand(60000, 47)
print(H.shape)
print(H)

(60000, 47)
[[0.73381412 0.25964245 0.85269754 ... 0.5485695  0.02970626 0.27021045]
 [0.0098423  0.4355777  0.97304208 ... 0.61285447 0.5801299  0.24818694]
 [0.24185742 0.71085661 0.695936   ... 0.18473198 0.89046182 0.46635063]
 ...
 [0.20104714 0.79819324 0.67521615 ... 0.33429603 0.85958191 0.50527446]
 [0.94129889 0.8031049  0.41771092 ... 0.26592046 0.00590372 0.68506177]
 [0.2255826  0.16537059 0.13502342 ... 0.55256423 0.91012608 0.38616573]]


1st Forward Learn

In [37]:
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(H, X)
print(reg.score(H, X))
X_pred = reg.predict(H)
print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.0008283244946358989
MAE 0.4444723959616089
MSE 0.9137832797669974
R2 -1146.9943616037758
R2 vw -1205.2563426682048
R2 ua -1146.9943616037758
(784, 47)
(784,)


1st Back Activations

In [38]:
XT = X.transpose()
print(f'shape of X_train {X.shape} and X_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

shape of X_train (60000, 784) and X_train transposed (784, 60000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.32662283320227303
MAE 0.41938725689437534
MSE 0.6029921665119466
R2 -0.7753276902430041
R2 vw -1.061635312497317
R2 ua -0.7753276902430041
(60000, 47)
(60000,)


2nd Forward learn - result are Coherent Weights

In [39]:
reg = LinearRegression().fit(H_new, X)
print(reg.score(H_new, X))
X_pred = reg.predict(H_new)
print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.45111437915228336
MAE 0.3439832698745408
MSE 0.5019783037599513
R2 -1.0174002672557898
R2 vw -0.21673270951614798
R2 ua -1.0174002672557898
(784, 47)
(784,)


2nd Back Activation - final representation of X_train in Hidden Activations

In [40]:
reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.4876921176771286
MAE 0.32747559144714194
MSE 0.4587587092566359
R2 vw -0.05047398502765832
R2 ua 0.1821958626258969
(60000, 47)
(60000,)


KNN test with Hidden Activations

In [41]:
# X = [[0], [1], [2], [3]]
# y = [0, 0, 1, 1]

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
X_p_test = X_test.reshape(len(X_test), -1).astype('float64')

# Dimensionen um den Mittelpunkt zentrieren
preproccessing = StandardScaler()
X_p_test = preproccessing.fit_transform(X_p_test)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_test.shape, np.mean(X_test), np.std(X_test)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_p_test.shape, np.mean(X_p_test), np.std(X_p_test)))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5, n_jobs=-1, weights='distance')
neigh.fit(H_new, y_train)

# print(neigh.predict([[1.1]]))

# print(neigh.predict_proba([[0.9]]))

Originaldaten:
Shape: (10000, 28, 28), Mean: 33.791224, STD: 79.172463
Vorbereitete Daten:
Shape: (10000, 784), Mean: 0.000000, STD: 0.923061


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='distance')

Back Activation of Test Data

In [42]:
XT = X_p_test.transpose()
print(f'shape of X_p_train {X_p_test.shape} and X_p_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_test = reg.coef_
print(H_test.shape)
HI_test = reg.intercept_
print(HI_test.shape)

shape of X_p_train (10000, 784) and X_p_train transposed (784, 10000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.5140096634008963
MAE 0.3235687078691093
MSE 0.4048574803435664
R2 0.21183028927938177
R2 vw 0.05451128411945735
R2 ua 0.21183028927938177
(10000, 47)
(10000,)


In [43]:
# Predicting the Test set results
y_pred = neigh.predict(H_test)

In [44]:
y_prob = neigh.predict_proba(H_test)

In [45]:
print(y_prob[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [48]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 966    0    0    2    1    5    3    3    0    0]
 [   0 1125    3    0    2    0    4    0    1    0]
 [   8    6  979    8    3    1    5    8   13    1]
 [   0    1    4  955    0   14    2   12   16    6]
 [   0    2    5    2  942    0    6    3    3   19]
 [   4    0    2   25    4  829   11    2    9    6]
 [   5    3    1    0    4    6  937    0    2    0]
 [   0   17   14    2    5    1    0  956    2   31]
 [   5    0    6   11    8   17    5    5  908    9]
 [   4    6    4   11   17    3    0   19    6  939]]


In [49]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9536
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.97      0.99      0.98      1135
           2       0.96      0.95      0.96      1032
           3       0.94      0.95      0.94      1010
           4       0.96      0.96      0.96       982
           5       0.95      0.93      0.94       892
           6       0.96      0.98      0.97       958
           7       0.95      0.93      0.94      1028
           8       0.95      0.93      0.94       974
           9       0.93      0.93      0.93      1009

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

